In [82]:
import pandas as pd
import re
import random

# ASTE

In [83]:
#train
train = pd.read_csv('Data/post-train/ASTE/train.txt', sep='.####', header=None, names=['input', 'label'])
train['label'] = train['label'].apply(lambda x:x[1:-1])
#test
test = pd.read_csv('Data/post-train/ASTE/dev.txt', sep='.####', header=None, names=['input', 'label'])
test['label'] = test['label'].apply(lambda x:x[1:-1])
#dev
dev = pd.read_csv('Data/post-train/ASTE/test.txt', sep='.####', header=None, names=['input', 'label'])
dev['label'] = dev['label'].apply(lambda x:x[1:-1])
dev.head()

C:\Users\danendra\AppData\Local\Temp\ipykernel_28324\2852429949.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train = pd.read_csv('Data/post-train/ASTE/train.txt', sep='.####', header=None, names=['input', 'label'])
C:\Users\danendra\AppData\Local\Temp\ipykernel_28324\2852429949.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test = pd.read_csv('Data/post-train/ASTE/dev.txt', sep='.####', header=None, names=['input', 'label'])
C:\Users\danendra\AppData\Local\Temp\ipykernel_28324\2852429949.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex se

,input,label
0,"pelayanan ramah , kamar nyaman dan fasilitas l...","([0], [1], 'POS'), ([3], [4], 'POS'), ([6], [7..."
1,tidak terlalu jauh dari pusat kota,"([3, 4, 5], [0, 1, 2], 'POS')"
2,dengan harga terjangkau kita sudah mendapatkan...,"([1], [2], 'POS'), ([6], [8], 'POS')"
3,"kondisinya cukup baik , hanya dapatnya tempat ...","([0], [1, 2], 'POS'), ([5, 6], [8, 9], 'NEG'),..."
4,"kamar bersih , bentuknya unik , tetapi sinyal ...","([0], [1], 'POS'), ([3], [4], 'POS'), ([7, 8],..."


In [85]:
train.shape

(3000, 2)

In [84]:
train.shape[0]+test.shape[0]+dev.shape[0]

5000

In [4]:
def convert_int(duplet):
    for d in range(len(duplet)):
        for i in range(len(duplet[d])):
            duplet[d][i]=int(duplet[d][i])
    return duplet

In [6]:
def convert_duplet(out):
    #untuk 1 output
    all_triplet = out.split("),")
    #find all in closed bracket
    duplet = [re.findall('\[(.*?)\]', triplet) for triplet in all_triplet]
    duplet = [[i.split(',') for i in d] for d in duplet]
    duplet = [convert_int(d) for d in duplet]
    #find all sentiment
    sentiment = [re.findall("\'.*?\'", triplet) for triplet in all_triplet]
    sentiment = [sent[0][1:-1] for sent in sentiment]
    return duplet, sentiment

In [7]:
#ubah angka ke kata
def convert_int_string(inp, duplet):
    inp_list = inp.split()
    for i in range(len(duplet)):
        for j in range(len(duplet[i])):
            for k in range(len(duplet[i][j])):
                duplet[i][j][k] = inp_list[duplet[i][j][k]]
    return duplet

In [8]:
def convert_triplet(inp, out):
    duplet, sentiment = convert_duplet(out)
    duplet = convert_int_string(inp, duplet)
    label = [duplet[i]+[sentiment[i]] for i in range(len(duplet))]
    #all triplet in triplets
    triplet_str_arr = []
    for i,triplet in enumerate(label):
        triplet_str = ""
        #get all tuple
        for j,tuple in enumerate(triplet):
            #get all aspect from the same tuple
            if type(tuple)==str:
                triplet_str+=tuple
            else:
                triplet_str+=" ".join(tuple)
            if j<(len(triplet)-1):
                triplet_str+=','
        triplet_str=f'({triplet_str})'
        if i<(len(label)-1):
            triplet_str+=';'
        triplet_str_arr.append(triplet_str)
    label = "".join(triplet_str_arr)
    return label

In [9]:
def transform_label(df, x, y):
    labels = []
    errors = []
    for i in range(len(df)):
        try:
            row = df.iloc[i]
            inp = row[x]
            out = row[y]
            triplet = convert_triplet(inp, out)
            labels.append(triplet)
        except:
            errors.append(i)
    return labels, errors

In [10]:
y_train, train_errors = transform_label(train, 'input', 'label')
y_test, test_errors = transform_label(test, 'input', 'label')
y_dev, dev_errors = transform_label(dev, 'input', 'label')

In [11]:
train = train.drop(train_errors)
test = test.drop(test_errors)
dev = dev.drop(dev_errors)

In [12]:
train['triplet'] = y_train
test['triplet'] = y_test
dev['triplet'] = y_dev

In [17]:
train.to_csv('Data/post-train/ASTE/train.csv', index=False)
test.to_csv('Data/post-train/ASTE/test.csv', index=False)
dev.to_csv('Data/post-train/ASTE/dev.csv', index=False)

In [15]:
pd.concat([train, test, dev]).to_csv('Data/post-train/ASTE/all_data.csv', index=False)

# TASD

In [124]:
#train
train = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/tasd/rest15/train.txt', sep='.####', header=None, names=['input', 'label'])
train['label'] = train['label'].apply(lambda x:x[1:-1])
#test
test = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/tasd/rest15/test.txt', sep='.####', header=None, names=['input', 'label'])
test['label'] = test['label'].apply(lambda x:x[1:-1])
#dev
dev = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/tasd/rest15/dev.txt', sep='.####', header=None, names=['input', 'label'])
dev['label'] = dev['label'].apply(lambda x:x[1:-1])
dev.head()

c:\Users\danendra\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,input,label
0,Judging from previous posts this used to be a ...,"('place', 'restaurant general', 'negative')"
1,"We , there were four of us , arrived at noon -...","('staff', 'service general', 'negative')"
2,"They never brought us complimentary noodles , ...","('NULL', 'service general', 'negative')"
3,The food was lousy - too sweet or too salty an...,"('food', 'food quality', 'negative'), ('portio..."
4,"After all that , they complained to me about t...","('NULL', 'service general', 'negative')"


In [128]:
tasd_df = pd.concat([train, test, dev])
tasd_df

,input,label
0,Judging from previous posts this used to be a ...,"('place', 'restaurant general', 'negative')"
1,"We , there were four of us , arrived at noon -...","('staff', 'service general', 'negative')"
2,"They never brought us complimentary noodles , ...","('NULL', 'service general', 'negative')"
3,The food was lousy - too sweet or too salty an...,"('food', 'food quality', 'negative'), ('portio..."
4,"After all that , they complained to me about t...","('NULL', 'service general', 'negative')"
...,...,...
5,Avoid this place,"('place', 'restaurant general', 'negative')"
6,"I have eaten at Saul , many times , the food i...","('food', 'food quality', 'positive')"
7,Saul is the best restaurant on Smith Street an...,"('Saul', 'restaurant general', 'positive')"
8,The duck confit is always amazing and the foie...,"('foie gras terrine with figs', 'food quality'..."


In [159]:
#aste data
aste_df = pd.read_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/aste/rest15/all_data.csv')
aste_df = aste_df.rename(columns={'label':'aste_label'})
aste_df

,input,aste_label,triplet
0,Judging from previous posts this used to be a ...,"([10], [9], 'NEG')","(place,good,NEG)"
1,"We , there were four of us , arrived at noon -...","([19], [31], 'NEG')","(staff,rude,NEG)"
2,The food was lousy - too sweet or too salty an...,"([1], [3], 'NEG'), ([1], [5, 6], 'NEG'), ([1],...","(food,lousy,NEG);(food,too sweet,NEG);(food,to..."
3,Avoid this place,"([2], [0], 'NEG')","(place,Avoid,NEG)"
4,"I have eaten at Saul , many times , the food i...","([10], [16], 'POS')","(food,good,POS)"
...,...,...,...
1070,The food is okay and the prices here are medio...,"([1], [3], 'NEU')","(food,okay,NEU)"
1071,Baluchi 's has solid food and a nice decor at ...,"([4], [3], 'POS'), ([8], [7], 'POS'), ([0, 1],...","(food,solid,POS);(decor,nice,POS);(Baluchi 's,..."
1072,$ 20 for all you can eat sushi can not be beaten,"([3, 4, 5, 6, 7], [11], 'POS')","(all you can eat sushi,beaten,POS)"
1073,"The vibe is very relaxed and cozy , service wa...","([1], [4], 'POS'), ([1], [6], 'POS'), ([8], [1...","(vibe,relaxed,POS);(vibe,cozy,POS);(service,gr..."


In [157]:
aste_df.shape

(1075, 3)

In [160]:
merged_df = aste_df.join(tasd_df.set_index('input'), on='input')
merged_df = merged_df[~merged_df.index.duplicated(keep='first')]
merged_df = merged_df.dropna()
merged_df

,input,aste_label,triplet,label
0,Judging from previous posts this used to be a ...,"([10], [9], 'NEG')","(place,good,NEG)","('place', 'restaurant general', 'negative')"
1,"We , there were four of us , arrived at noon -...","([19], [31], 'NEG')","(staff,rude,NEG)","('staff', 'service general', 'negative')"
2,The food was lousy - too sweet or too salty an...,"([1], [3], 'NEG'), ([1], [5, 6], 'NEG'), ([1],...","(food,lousy,NEG);(food,too sweet,NEG);(food,to...","('food', 'food quality', 'negative'), ('portio..."
3,Avoid this place,"([2], [0], 'NEG')","(place,Avoid,NEG)","('place', 'restaurant general', 'negative')"
4,"I have eaten at Saul , many times , the food i...","([10], [16], 'POS')","(food,good,POS)","('food', 'food quality', 'positive')"
...,...,...,...,...
1068,"whoever the jazz duo was , they were on POINT","([2, 3], [8, 9], 'POS')","(jazz duo,on POINT,POS)","('jazz duo', 'ambience general', 'positive')"
1069,even the wine by the glass was good,"([2, 3, 4, 5], [7], 'POS')","(wine by the glass,good,POS)","('wine by the glass', 'drinks quality', 'posit..."
1070,The food is okay and the prices here are medio...,"([1], [3], 'NEU')","(food,okay,NEU)","('food', 'food quality', 'neutral'), ('NULL', ..."
1071,Baluchi 's has solid food and a nice decor at ...,"([4], [3], 'POS'), ([8], [7], 'POS'), ([0, 1],...","(food,solid,POS);(decor,nice,POS);(Baluchi 's,...","('food', 'food quality', 'positive'), ('decor'..."


In [181]:
idx = random.randint(0, len(merged_df))
row = merged_df.iloc[idx]
inp = row['input']
triplet = row['triplet']
aste_label = row['aste_label']
label = row['label']
print(inp)
print(aste_label)
print(triplet)
print(label)
all_categories = label.split(')')
#all_categories

Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening 
([4], [3], 'POS'), ([7], [6], 'POS')
(service,great,POS);(food,great,POS)
('service', 'service general', 'positive'), ('food', 'food quality', 'positive'), ('food', 'food prices', 'positive')


In [203]:
def extract_pairs(label):
    """
    get all pairs from TASD label, the pair is
    (aspect, aspect category)
    """
    all_tasd = re.findall('\[(.*?)\]', label.replace('(','[').replace(')',']').replace("'",''))
    all_tasd = [tasd.split(',') for tasd in all_tasd]
    all_pair = [(tasd[0].strip(), tasd[1].strip()) for tasd in all_tasd]
    return all_pair

In [268]:
def convert_to_quadruplet(pairs, triplets):
    triplets = triplets.split(';')
    triplets = [triplet[1:-1].split(',') for triplet in triplets]
    quadruplets = []
    #triplet_list[0] is aspect
    for triplet in triplets:
        aspect = triplet[0]
        quadruplet=[]
        for pair in pairs:
            #if aspect in pair from TASD is the same as 
            #aspect in triplet from ASTE then combine it
            if pair[0]==aspect:
                quadruplet = triplet+[pair[1]]
        #if there are no same aspect from TASD in ASTE that means
        #we will drop the triplet
        if len(quadruplet)>=4:
            quadruplets.append(quadruplet)
    return quadruplets
        

In [260]:
def string_quadruplet(quadruplets):
    quadruplets_str = ''
    for i,quadruplet in enumerate(quadruplets):
        quadruplet_str = f'({str(quadruplet)})'
        quadruplets_str+=quadruplet_str.replace('[','').replace(']','').replace("'",'')
        if i<(len(quadruplets)-1):
            quadruplets_str+=';'
    return quadruplets_str

In [273]:
#idx = random.randint(0, len(merged_df))
idx=305
row = merged_df.iloc[idx]
inp = row['input']
triplet = row['triplet']
aste_label = row['aste_label']
label = row['label']
print(inp)
print(triplet)
print(label)
#pair
all_pair = extract_pairs(label)
#quadruplet
quadruplet = convert_to_quadruplet(all_pair, triplet)
quadruplet = string_quadruplet(quadruplet)
print(quadruplet)

An excellent servic
(servic,excellent,POS)
('service', 'service general', 'positive')



In [285]:
def transform_to_quadruplet(df, triplet_col, tasd_col):
    quadruplets = []
    for i in range(len(df)):
        row = df.iloc[i]
        triplet = row[triplet_col]
        label = row[tasd_col]
        #pair
        all_pair = extract_pairs(label)
        #quadruplet
        quadruplet = convert_to_quadruplet(all_pair, triplet)
        quadruplet = string_quadruplet(quadruplet)
        quadruplets.append(quadruplet)
    return quadruplets

In [287]:
y = transform_to_quadruplet(merged_df, 'triplet', 'label')
y

['(place, good, NEG, restaurant general)',
 '(staff, rude, NEG, service general)',
 '(food, lousy, NEG, food quality);(food, too sweet, NEG, food quality);(food, too salty, NEG, food quality);(portions, tiny, NEG, food style_options)',
 '(place, Avoid, NEG, restaurant general)',
 '(food, good, POS, food quality)',
 '(Saul, best, POS, restaurant general)',
 '(foie gras terrine with figs, out of this world, POS, food quality);(duck confit, amazing, POS, food quality)',
 '(wine list, interesting, POS, drinks prices);(wine list, good values, POS, drinks prices)',
 '(restaurant, disappointed, NEG, restaurant general)',
 '(Food, okay, NEU, food quality);(Food, nothing great, NEU, food quality)',
 '(service, excellent, POS, service general);(decor, cool, POS, ambience general);(decor, understated, POS, ambience general)',
 '(duck breast special, incredible, POS, food quality)',
 '(Grilled Chicken special with Edamame Puree, enjoyed, POS, food quality)',
 '(food, decent, NEU, food quality)',
 

In [288]:
quadruplet_df = merged_df.drop(columns=['aste_label'])
quadruplet_df['quadruplet_label'] = y


In [293]:
quadruplet_df[quadruplet_df['quadruplet_label']!=''].to_csv('D:/Kuliah/S2/Tesis/Code/Generative-ABSA-main/data/quadruplet/rest15/all_data.csv', index=False)

In [292]:
quadruplet_df

,input,triplet,label,quadruplet_label
0,Judging from previous posts this used to be a ...,"(place,good,NEG)","('place', 'restaurant general', 'negative')","(place, good, NEG, restaurant general)"
1,"We , there were four of us , arrived at noon -...","(staff,rude,NEG)","('staff', 'service general', 'negative')","(staff, rude, NEG, service general)"
2,The food was lousy - too sweet or too salty an...,"(food,lousy,NEG);(food,too sweet,NEG);(food,to...","('food', 'food quality', 'negative'), ('portio...","(food, lousy, NEG, food quality);(food, too sw..."
3,Avoid this place,"(place,Avoid,NEG)","('place', 'restaurant general', 'negative')","(place, Avoid, NEG, restaurant general)"
4,"I have eaten at Saul , many times , the food i...","(food,good,POS)","('food', 'food quality', 'positive')","(food, good, POS, food quality)"
...,...,...,...,...
1068,"whoever the jazz duo was , they were on POINT","(jazz duo,on POINT,POS)","('jazz duo', 'ambience general', 'positive')","(jazz duo, on POINT, POS, ambience general)"
1069,even the wine by the glass was good,"(wine by the glass,good,POS)","('wine by the glass', 'drinks quality', 'posit...","(wine by the glass, good, POS, drinks quality)"
1070,The food is okay and the prices here are medio...,"(food,okay,NEU)","('food', 'food quality', 'neutral'), ('NULL', ...","(food, okay, NEU, food quality)"
1071,Baluchi 's has solid food and a nice decor at ...,"(food,solid,POS);(decor,nice,POS);(Baluchi 's,...","('food', 'food quality', 'positive'), ('decor'...","(food, solid, POS, food quality);(decor, nice,..."


# Quadruplet Manual Anotation

In [30]:
df = pd.read_csv('data/quadruplet_test_pred.csv')
df = df.rename(
    columns={
        'content' : 'original_tweet',
        'final_sentiment' : 'nolimit_sentiment',
        'labels' : 'nolimit_aspect_categories',
        'sentiment_label' : 'annotated_spam',
        'preds' : 'model_spam'
    }
)
df['quadruplet'] = df['quadruplet'].apply(lambda x:x[1:-1])
df = df.drop(columns=['pred_quadruplet_pt_t5', 'pred_quadruplet_tf_t5', 'spam'])
df.head()

,original_id,original_tweet,clean_tweet,nolimit_sentiment,nolimit_aspect_categories,quadruplet,annotated_spam,model_spam
0,1.640000e+18,@loseratlover @TokopediaCare @gojekindonesia @...,"Sering di toko ini dibawa kabur kurir, temen s...",negative,delivery; produk;,"(toko, sering dibawa kabur kurir, negative, de...",2,2
1,1.640000e+18,@bukugpu @fiksigpu @sastragpu @bincang_buku @K...,Min Funicula di Tokped belum diskon %. Tolong ...,negative,price; produk;,"(Funicula, belum diskon, neutral, price);",2,2
2,1.640000e+18,admin blibli kocak juga,admin blibli kocak juga,neutral,customerservice; produk;,"(admin blibli, kocak, positive, customerservice);",2,2
3,1.640000e+18,@tokopedia upload video untuk komplain lama se...,"upload video untuk komplain lama sekali, mana ...",negative,website&apps; produk;,"(upload video, lama sekali, negative, website&...",2,2
4,1.640000e+18,Kalo barang ga sesuai bisa diajukan pengembali...,Kalo barang ga sesuai bisa diajukan pengembali...,negative,payment; produk;,"(barang, ga sesuai, neutral, payment);",2,2


In [24]:
df.to_csv('data/quadruplet_only.csv', index=False)

# Post train data

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizerFast
from transformers import BertForSequenceClassification
import preprocessor as p
from transformers import TrainingArguments
from transformers import Trainer
from transformers import DataCollatorWithPadding
import torch
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
import os

In [44]:
tokenizer = BertTokenizerFast.from_pretrained('indolem/indobert-base-uncased')
id2label = {0: "SPAM", 1: "OBJECTIVE", 2:"SUBJECTIVE"}
label2id = {"SPAM": 0, "OBJECTIVE": 1, "SUBJECTIVE":2}
model = BertForSequenceClassification.from_pretrained('D:/Kuliah/S2/Tesis/Code/spam_detection/test-trainer/checkpoint-1770'
                                                      , num_labels=3, id2label=id2label, label2id=label2id).to('cuda')


In [68]:
csv_files

['Ecommerce_Indonesia_01-Dec-2022_31-Dec-2022_TIJA1y16tr_0.csv',
 'Ecommerce_Indonesia_01-Dec-2022_31-Dec-2022_TIJA1y16tr_1.csv',
 'Ecommerce_Indonesia_01-Dec-2022_31-Dec-2022_TIJA1y16tr_2.csv',
 'Ecommerce_Indonesia_01-Dec-2022_31-Dec-2022_TIJA1y16tr_3.csv',
 'Ecommerce_Indonesia_01-Dec-2022_31-Dec-2022_TIJA1y16tr_4.csv',
 'Ecommerce_Indonesia_01-Jan-2023_31-Jan-2023_gtJEUiS9JH_0.csv',
 'Ecommerce_Indonesia_01-Jan-2023_31-Jan-2023_gtJEUiS9JH_1.csv',
 'Ecommerce_Indonesia_01-Jan-2023_31-Jan-2023_gtJEUiS9JH_2.csv',
 'Ecommerce_Indonesia_01-Jan-2023_31-Jan-2023_gtJEUiS9JH_3.csv',
 'Ecommerce_Indonesia_01-Jan-2023_31-Jan-2023_gtJEUiS9JH_4.csv',
 'Ecommerce_Indonesia_01-Nov-2022_30-Nov-2022_wjV3n9Ppve_0.csv',
 'Ecommerce_Indonesia_01-Nov-2022_30-Nov-2022_wjV3n9Ppve_1.csv',
 'Ecommerce_Indonesia_01-Nov-2022_30-Nov-2022_wjV3n9Ppve_2.csv',
 'Ecommerce_Indonesia_01-Nov-2022_30-Nov-2022_wjV3n9Ppve_3.csv',
 'Ecommerce_Indonesia_01-Nov-2022_30-Nov-2022_wjV3n9Ppve_4.csv',
 'Ecommerce_Indonesia_01-

In [69]:
file_path = f'{post_train_folder}/{csv_files[0]}'
df = pd.read_csv(file_path, sep=';')
clean_df = df[(df['post_type']=="'talk")& #karena yg post
                (df['lang']=="'in")&
                (df['content_type']=="'text")
            ]
clean_df['clean_tweet'] = clean_df['content'].apply(p.clean)
clean_df = clean_df.drop_duplicates(subset=['clean_tweet'])
clean_df = clean_df.reset_index(drop=True)
clean_df = clean_df['clean_tweet'].to_frame()

C:\Users\danendra\AppData\Local\Temp\ipykernel_30504\1798584822.py:2: DtypeWarning: Columns (52,58,65,68,70,71,72,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,130,131,132,133,134,135,136,137,138,139,140,141,142,143,145,147,148,149,150,151,152,153,154,155,156,157,158,159,160,162,163,164,167,168,169,170,171,172,173,174,175,177,178,179,180,181,182,183,184,185,187,188,191,192,194,195,196,197,198,199,200,201,202,203,204,206,207,208,209,210,212,213,214,216,217,218,219,220,221,223,224,225,229,232,233,234,237,238,239,243,244,245,247,248,251,252,254,256,257,261,262,263,264,266,267,268,270,271,273,274,275,277,278,279,280,282,284,286,287,289,291,292,293,295,296,298,299,301,303,304,305,306,307,308,309,310,311,312,313,315,321,322,327,328,329,330,331,333,334,338,342,345,347,350,351,352,353,355,357,358,359,365,366,367,368,369,371,372,373,375,376,377,379,380,381,382,

In [70]:
clean_df.shape

(26690, 1)

In [64]:
post_train_folder = 'Data/post-train/'
csv_files = [file for file in os.listdir(post_train_folder) if file.endswith('.csv')]
for file_idx, file in enumerate(csv_files):
    print(file_idx)
    file_path = f'{post_train_folder}/{file}'
    df = pd.read_csv(file_path, sep=';')
    clean_df = df[(df['post_type']=="'talk")& #karena yg post
                    (df['lang']=="'in")&
                    (df['content_type']=="'text")
                ]
    clean_df['clean_tweet'] = clean_df['content'].apply(p.clean)
    clean_df = clean_df.drop_duplicates(subset=['clean_tweet'])
    clean_df = clean_df.reset_index(drop=True)
    clean_df = clean_df['clean_tweet'].to_frame()
    clean_dataset = Dataset.from_pandas(clean_df)
    #prediction
    classifier = pipeline(task='text-classification',
                        tokenizer=tokenizer, 
                        model=model, device='cuda:0')
    labels = []
    for out in tqdm(classifier(KeyDataset(clean_dataset, "clean_tweet"))):
        labels.append(out['label'])
    clean_df['tweet_type'] = labels
    clean_df = clean_df[clean_df['tweet_type']!="SPAM"]
    clean_df = clean_df.reset_index(drop=True)
    #save to file
    clean_file_path = f'{post_train_folder}/clean_tweet.csv'
    #klo ada buat kita append
    if os.path.isfile(clean_file_path):
        text_df = pd.read_csv(clean_file_path)
        text_df = pd.concat([text_df, clean_df])
        print(text_df.shape)
        text_df.to_csv(clean_file_path, index=False)
    #kalau gk ada maka buat baru
    else:
        clean_df.to_csv(clean_file_path, index=False)

0


C:\Users\danendra\AppData\Local\Temp\ipykernel_30504\96602400.py:6: DtypeWarning: Columns (52,58,65,68,70,71,72,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,130,131,132,133,134,135,136,137,138,139,140,141,142,143,145,147,148,149,150,151,152,153,154,155,156,157,158,159,160,162,163,164,167,168,169,170,171,172,173,174,175,177,178,179,180,181,182,183,184,185,187,188,191,192,194,195,196,197,198,199,200,201,202,203,204,206,207,208,209,210,212,213,214,216,217,218,219,220,221,223,224,225,229,232,233,234,237,238,239,243,244,245,247,248,251,252,254,256,257,261,262,263,264,266,267,268,270,271,273,274,275,277,278,279,280,282,284,286,287,289,291,292,293,295,296,298,299,301,303,304,305,306,307,308,309,310,311,312,313,315,321,322,327,328,329,330,331,333,334,338,342,345,347,350,351,352,353,355,357,358,359,365,366,367,368,369,371,372,373,375,376,377,379,380,381,382,38

1


C:\Users\danendra\AppData\Local\Temp\ipykernel_30504\96602400.py:6: DtypeWarning: Columns (52,54,58,65,68,70,71,72,75,76,77,78,79,80,81,82,83,84,85,87,88,89,90,91,92,93,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,130,131,132,133,134,135,136,137,138,139,140,142,143,145,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,185,186,187,188,190,191,192,194,195,196,197,198,200,202,203,204,206,207,208,209,210,211,212,213,214,216,217,218,219,223,224,225,229,232,233,234,238,239,241,243,245,247,248,249,251,252,253,254,255,257,258,261,262,263,266,267,269,270,274,275,277,278,279,280,281,282,284,286,288,289,291,292,293,296,298,299,300,301,302,303,304,305,306,307,309,310,311,312,313,319,321,322,325,327,328,329,330,333,334,335,338,339,342,343,345,346,347,348,350,351,352,353,355,357,358,359,365,366,367,368,371,372,373,376,377,379

(18710, 2)
2


...

(78044, 2)
14


C:\Users\danendra\AppData\Local\Temp\ipykernel_30504\96602400.py:6: DtypeWarning: Columns (15,51,52,53,58,62,65,67,68,70,71,72,74,75,77,79,80,83,84,85,87,89,90,93,94,97,98,99,102,103,104,106,111,112,113,115,116,119,121,123,124,125,127,128,131,134,135,138,140,142,143,147,148,149,152,154,158,159,160,163,167,168,169,173,174,175,177,178,181,182,183,184,185,187,188,191,192,195,198,202,203,204,208,209,212,213,214,218,219,223,232,234,238,243,254,257,264,266,267,270,278,279,280,281,282,286,291,292,293,298,299,304,306,312,322,328,329,330,331,333,334,335,342,347,350,352,353,358,359,367,368,372,373,385,387,389,391,396,397,404,405,406,407,409,413,414,415,417,419,420,422,424,426,428,430,432,440,442,444,446,448,451,455,456,457,458,459,460,461,462,463,464,465,469,470,471,472,475,476,479,480,481,482,486,488,489,490,492,495,496,498,499,501,503,504,505,506,508,509,510,511,512,513,516,519,520,521,522,523,525,526,527,530,532,533,534,535,536,538,540,543,545,550,556,560,567,571,572,573,574,575,577,578,581,5

(83184, 2)
15


C:\Users\danendra\AppData\Local\Temp\ipykernel_30504\96602400.py:6: DtypeWarning: Columns (51,53,65,67,147,163,168,169,178,182,185,195,219,243,266,291,322,335,350,406,440,459,476,483,503,504,510,571,578,597,640,641,642,643,675,711,713) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';')
C:\Users\danendra\AppData\Local\Temp\ipykernel_30504\96602400.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['clean_tweet'] = clean_df['content'].apply(p.clean)
100%|██████████| 713/713 [00:05<00:00, 121.45it/s]


(83415, 2)


In [65]:
test_df = pd.read_csv(clean_file_path)
test_df.shape

(83415, 2)

# Quadruplet 4k data

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizerFast
from transformers import BertForSequenceClassification
import preprocessor as p
from transformers import TrainingArguments
from transformers import Trainer
from transformers import DataCollatorWithPadding
import torch
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
post_detail = [
    'original_id',
    'date_created',
    'specific_type',
    'post_type',
    'object_name',
    'content',
    'final_sentiment',
    'lang',
    'content_type'
] 
post_stats = [
    'followers',
    'share',
    'like',
    'comment',
    'engagement',
    'reach',
    'impression',
    'engagement_rate'
]
labels = [
    'delivery',
    'customerservice',
    'website&apps',
    'payment',
    'price',
    'produk'
]
used_col = post_detail+post_stats+labels

In [3]:
#df1 = pd.read_csv('data/Ecommerce_Indonesia_20-Mar-2023_20-Apr-2023_GB4eugwhcl/Ecommerce_Indonesia_20-Mar-2023_20-Apr-2023_GB4eugwhcl_0.csv', sep=';')
#df2 = pd.read_csv('data/quadruplet/Ecommerce_Indonesia_20-Mar-2023_20-Apr-2023_GB4eugwhcl/Ecommerce_Indonesia_20-Mar-2023_20-Apr-2023_GB4eugwhcl_1.csv', sep=';')
df = pd.read_csv("Data/post-train/MLM/Ecommerce_Indonesia_01-Dec-2022_31-Dec-2022_TIJA1y16tr_0.csv", sep=';')

C:\Users\danendra\AppData\Local\Temp\ipykernel_12632\3437453336.py:3: DtypeWarning: Columns (52,58,65,68,70,71,72,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,130,131,132,133,134,135,136,137,138,139,140,141,142,143,145,147,148,149,150,151,152,153,154,155,156,157,158,159,160,162,163,164,167,168,169,170,171,172,173,174,175,177,178,179,180,181,182,183,184,185,187,188,191,192,194,195,196,197,198,199,200,201,202,203,204,206,207,208,209,210,212,213,214,216,217,218,219,220,221,223,224,225,229,232,233,234,237,238,239,243,244,245,247,248,251,252,254,256,257,261,262,263,264,266,267,268,270,271,273,274,275,277,278,279,280,282,284,286,287,289,291,292,293,295,296,298,299,301,303,304,305,306,307,308,309,310,311,312,313,315,321,322,327,328,329,330,331,333,334,338,342,345,347,350,351,352,353,355,357,358,359,365,366,367,368,369,371,372,373,375,376,377,379,380,381,382,

In [4]:
df = df[used_col]
clean_df = df[(df['post_type']=="'talk")& #karena yg post
                    (df['lang']=="'in")&
                    (df['content_type']=="'text")
                    ]

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('indolem/indobert-base-uncased')
id2label = {0: "SPAM", 1: "OBJECTIVE", 2:"SUBJECTIVE"}
label2id = {"SPAM": 0, "OBJECTIVE": 1, "SUBJECTIVE":2}
model = BertForSequenceClassification.from_pretrained('D:/Kuliah/S2/Tesis/Code/spam_detection/test-trainer/checkpoint-1770'
                                                      , num_labels=3, id2label=id2label, label2id=label2id)


In [7]:
def clean_data(examples):
    clean_tweets = [p.clean(text) for text in examples['content']]
    examples['clean_tweet'] = clean_tweets
    return examples

In [8]:
dataset = Dataset.from_pandas(clean_df)
clean_dataset = dataset.map(clean_data, batched=True, remove_columns=dataset.column_names)

In [11]:
classifier = pipeline(task='text-classification',
                      tokenizer=tokenizer, 
                      model=model, device='cuda:0')

In [12]:
labels = []
for out in tqdm(classifier(KeyDataset(clean_dataset, "clean_tweet"))):
    labels.append(out['label'])

100%|██████████| 78447/78447 [09:58<00:00, 131.16it/s]


In [13]:
clean_df['tweet_type'] = labels
clean_df['clean_tweet'] = clean_dataset['clean_tweet']

C:\Users\danendra\AppData\Local\Temp\ipykernel_12632\872735425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['tweet_type'] = labels
C:\Users\danendra\AppData\Local\Temp\ipykernel_12632\872735425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['clean_tweet'] = clean_dataset['clean_tweet']


In [14]:
clean_df = clean_df.drop_duplicates(subset=['clean_tweet'])
clean_df = clean_df.reset_index(drop=True)

In [15]:
#nyatuin label
aspect_categories = [
    'delivery',
    'customerservice',
    'website&apps',
    'payment',
    'price',
    'produk'
]
clean_df['baseline_aspect_category'] = ''
for i in tqdm(range(len(clean_df))):
    category_concat = ''
    row = clean_df.iloc[i]
    for cat in aspect_categories:
        if type(row[cat])==str:
            category_concat+=f'{cat}; '
    clean_df.at[i, 'baseline_aspect_category'] = category_concat

100%|██████████| 26690/26690 [00:01<00:00, 14064.03it/s]


In [30]:
#clean_df[['original_id', 'content', 'final_sentiment', 'baseline_aspect_category', 'tweet_type', 'clean_tweet']].to_csv('Data/Ecommerce_01-Dec-2022_31-Dec-2022_spam_pred.csv', index=False)

In [16]:
sentiment_only = clean_df[clean_df['tweet_type']=='SUBJECTIVE']
sentiment_only = sentiment_only.drop_duplicates(subset=['clean_tweet'])
sentiment_only = sentiment_only.reset_index(drop=True)

In [17]:
#nyatuin label
aspect_categories = [
    'delivery',
    'customerservice',
    'website&apps',
    'payment',
    'price',
    'produk'
]
sentiment_only['baseline_aspect_category'] = ''
for i in tqdm(range(len(sentiment_only))):
    category_concat = ''
    row = sentiment_only.iloc[i]
    for cat in aspect_categories:
        if type(row[cat])==str:
            category_concat+=f'{cat}; '
    sentiment_only.at[i, 'baseline_aspect_category'] = category_concat

100%|██████████| 5172/5172 [00:00<00:00, 13925.85it/s]


In [18]:
sentiment_only

,original_id,date_created,specific_type,post_type,object_name,content,final_sentiment,lang,content_type,followers,...,engagement_rate,delivery,customerservice,website&apps,payment,price,produk,tweet_type,clean_tweet,baseline_aspect_category
0,'1609093028043710464,31/12/2022 14.44,'status,'talk,'olx,'@worksfess Coba olx nder,'neutral,'in,'text,94,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,SUBJECTIVE,' Coba olx nder,
1,'1609024284567166978,31/12/2022 10.10,'status,'talk,'olx,'Iki olx keneng opo kok eror ki. Mata pencaha...,'negative,'in,'text,2,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,SUBJECTIVE,'Iki olx keneng opo kok eror ki. Mata pencahar...,
2,'1608428182386135041,29/12/2022 18.42,'status,'talk,'olx,'@winrte Kok bisa sih kamu punya?! Aku mau sat...,'negative,'in,'text,510,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,SUBJECTIVE,' Kok bisa sih kamu punya?! Aku mau satu juga ...,
3,'1608394553438142465,29/12/2022 16.28,'status,'talk,'olx,'@kegblgnunfaedh Pacar kayak gitu dijual aja d...,'negative,'in,'text,3,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,SUBJECTIVE,"' Pacar kayak gitu dijual aja di OLX, gaakan l...",
4,'1608393834664427520,29/12/2022 16.25,'status,'talk,'olx,"'kalo gua lagi gedeg bgt kerja disini, gua sel...",'neutral,'in,'text,2,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,SUBJECTIVE,"'kalo gua lagi gedeg bgt kerja disini, gua sel...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,'1606590558595878913,24/12/2022 17.00,'status,'talk,'shopee,'Apesi shopee halaman bermasalah mulu kesel gue,'neutral,'in,'text,1118,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,SUBJECTIVE,'Apesi shopee halaman bermasalah mulu kesel gue,
5168,'1606590472562315264,24/12/2022 16.59,'status,'talk,'shopee,'@Askrlfess Ga pernah hhh ngomong sama aku pal...,'negative,'in,'text,71,...,0.0,'Delivery,NaN,NaN,NaN,NaN,NaN,SUBJECTIVE,' Ga pernah hhh ngomong sama aku paling cuma n...,delivery;
5169,'1608489841892954114,29/12/2022 22.47,'status,'talk,'blibli,'@bliblidotcom Kwetiau ngliat aja udah eneg,'neutral,'in,'text,644,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,SUBJECTIVE,' Kwetiau ngliat aja udah eneg,
5170,'1608488137587830784,29/12/2022 22.40,'status,'talk,'blibli,'@bliblidotcom ES POTONG AJDIQBDJQJQ,'neutral,'in,'text,2,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,SUBJECTIVE,' ES POTONG AJDIQBDJQJQ,


In [19]:
sentiment_only[['original_id', 'content', 'final_sentiment', 'baseline_aspect_category', 'tweet_type', 'clean_tweet']].to_csv('Data/Ecommerce_01-Dec-2022_31-Dec-2022_sentiment_only_2.csv', index=False)

# Quadruplet Zili anotation
yang dianotasi zili 

In [1]:
import pandas as pd
from experiments.src.utils import extract_quadruplet
import re

In [2]:
data_1000 = 'Data/quadruplet/quadruplet_annottated_sample_dataset.csv'
data_2000 = 'Data/quadruplet/quadruplet_2200_data.csv'

In [17]:
df = pd.read_csv(data_2000)
clean_df = df.dropna(subset=['quadruplet'])
clean_df = clean_df.drop(columns=['Total', 'original_id'])
clean_df['quadruplet'] = clean_df['quadruplet'].apply(lambda x:x[:-1] if x[-1]==';' else x)
clean_df = clean_df.reset_index(drop=True)

In [18]:
clean_df.shape

(2337, 6)

In [19]:
error_quad = []
for i in range(len(clean_df)):
    extractions = extract_quadruplet(clean_df.iloc[i]['quadruplet'])
    for quad in extractions:
        if "" in quad:
            error_quad.append(i)

In [20]:
clean_df = clean_df.drop(index=error_quad).reset_index(drop=True)

In [21]:
clean_df.shape

(2313, 6)

In [22]:
def get_triplet(quadruplets):
    triplets = []
    for quad in quadruplets:
        aspect_term, opinion_term, sentiment, aspect_categories = quad
        triplets.append((aspect_term, opinion_term, sentiment))
    triplet_str = ''
    for i, triplet in enumerate(triplets):
        triplet_str+=str(triplet)
        if i<len(triplets)-1:
            triplet_str+=';'
    return triplet_str.replace("'", "")

In [23]:
all_triplets = []
errors_idx = []
for i in range(len(clean_df)):
    row = clean_df.iloc[i]
    quadruplets = extract_quadruplet(row['quadruplet'])
    triplets = get_triplet(quadruplets)
    if triplets!='':
        all_triplets.append(triplets)
    else:
        errors_idx.append(i)

In [24]:
clean_df = clean_df.drop(index=errors_idx).reset_index(drop=True)

In [25]:
clean_df.shape

(2298, 6)

In [26]:
clean_df['triplet_label'] = all_triplets

In [27]:
clean_df[['quadruplet', 'triplet_label']].head(10)

,quadruplet,triplet_label
0,"(hp, harga terlalu tinggi, negative, produk)","(hp, harga terlalu tinggi, negative)"
1,"(motor bekas, pertama kali, positive, produk)","(motor bekas, pertama kali, positive)"
2,"(beli second, biar lebih aman, negative, produ...","(beli second, biar lebih aman, negative);(olx,..."
3,"(nasi goreng, enak, positive, produk)","(nasi goreng, enak, positive)"
4,"(olx, reseller, negative, customerservice)","(olx, reseller, negative)"
5,"(olx, ditawar gila kali, negative, produk)","(olx, ditawar gila kali, negative)"
6,"(olx, ada manusiawinya, positive, produk); (fb...","(olx, ada manusiawinya, positive);(fb, kebanya..."
7,"(kostan, kafe dsana banyak, positive, produk);...","(kostan, kafe dsana banyak, positive);(google ..."
8,"(seller kandang kucing, nyebelin, negative, pr...","(seller kandang kucing, nyebelin, negative);(o..."
9,"(jual hp, harga murah, positive, produk); (cpt...","(jual hp, harga murah, positive);(cpt banget l..."


In [28]:
clean_df = clean_df.reset_index(drop=True)

In [30]:
clean_df.to_csv('Data/quadruplet/quadruplet_2200-data_annottated_clean.csv', index=False)

In [31]:
clean_df

,content,final_sentiment,baseline_aspect_category,tweet_type,clean_tweet,quadruplet,triplet_label
0,@JUALAN_BASE @fleurlovincs Nder hpnya kamu pak...,neutral,delivery; produk;,SUBJECTIVE,Nder hpnya kamu pake brp lama? Kayaknya jt ms...,"(hp, harga terlalu tinggi, negative, produk)","(hp, harga terlalu tinggi, negative)"
1,@teh_manis__ @ezash coba cari di olx mba kalo ...,neutral,produk;,SUBJECTIVE,coba cari di olx mba kalo baru pertama kali m...,"(motor bekas, pertama kali, positive, produk)","(motor bekas, pertama kali, positive)"
2,@tanyakanrl Kalo mau beli second jangan di tok...,positive,delivery; produk;,SUBJECTIVE,Kalo mau beli second jangan di toko oren atau...,"(beli second, biar lebih aman, negative, produ...","(beli second, biar lebih aman, negative);(olx,..."
3,kalo jam 1 malem gabisa tidur pengen makan nas...,neutral,customerservice; produk;,SUBJECTIVE,kalo jam malem gabisa tidur pengen makan nasi ...,"(nasi goreng, enak, positive, produk)","(nasi goreng, enak, positive)"
4,"@tirta_cipeng Di luar dunia fashion, saya seri...",neutral,produk;,SUBJECTIVE,"Di luar dunia fashion, saya sering banget nem...","(olx, reseller, negative, customerservice)","(olx, reseller, negative)"
...,...,...,...,...,...,...,...
2293,@indomilkyourway @alfamart @alfamidi_ku @Indom...,positive,produk;,SUBJECTIVE,PINK BLOSSOM ENAK BGTIIII,"(pink blossom, enak bgt, positive, produk)","(pink blossom, enak bgt, positive)"
2294,shopee gw doang apa emang lagi error sih. tiap...,negative,produk;,SUBJECTIVE,shopee gw doang apa emang lagi error sih. tiap...,"(shopee, lagi error, negative, website&apps)","(shopee, lagi error, negative)"
2295,@indomilkyourway @alfamart @alfamidi_ku @Indom...,positive,produk;,SUBJECTIVE,"Yang black latte enak banget sumpilll, plis j...","(black latte, enak banget sumpilll, positive, ...","(black latte, enak banget sumpilll, positive)"
2296,@sbtcon Di deskripsi link Shopee sllu tulis ka...,negative,produk;,SUBJECTIVE,Di deskripsi link Shopee sllu tulis kalo rusa...,"(packing, diluar tanggung jawab seller, negati...","(packing, diluar tanggung jawab seller, negative)"


# 1k Data quadruplet relabeling

In [1]:
import pandas as pd
import preprocessor as p
from experiments.src.postprocessor import PostProcessor
from experiments.src.utils import extract_quintuplet

In [3]:
df = pd.read_csv('Data/quintuplet/quintuple_data_clean.csv')
df = df.dropna(subset=['content'])
df['clean_tweet'] = df['content'].apply(p.clean)
df['clean_tweet'] = df['clean_tweet'].apply(str.lower)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2013 entries, 0 to 2012
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   aoriginal_id              2013 non-null   int64 
 1   content                   2013 non-null   object
 2   final_sentiment           2013 non-null   object
 3   baseline_aspect_category  1985 non-null   object
 4   tweet_type                2013 non-null   object
 5   clean_tweet               2013 non-null   object
 6   label                     2003 non-null   object
 7   Unnamed: 7                182 non-null    object
 8   Unnamed: 8                53 non-null     object
dtypes: int64(1), object(8)
memory usage: 141.7+ KB


bersih2 aspect category dan sentiment

In [4]:
errors_quad = []
for i in range(len(df)):
    row = df.iloc[i]
    quintuplets = extract_quintuplet(row['label'])
    for quintuplet in quintuplets:
        entity, aspect_term, opinion_term, sentiment, aspect_category = quintuplet
        if entity=='' or aspect_term=='' or opinion_term=='' or sentiment=='' or aspect_category=='':
            print(i, quintuplet)
            errors_quad.append(i)

In [5]:
errors_quad

[]

In [6]:
postprocessor = PostProcessor(use_postprocess=True)

In [7]:
df['postprocess_quintuplet']=''
for i in range(len(df)):
    row = df.iloc[i]
    orig_sentence = row['clean_tweet']
    clean_quintuplets = []
    quintuplets = extract_quintuplet(row['label'])
    for quintuplet in quintuplets:
        entity, aspect_term, opinion_term, sentiment, aspect_category = quintuplet
        entity, aspect_term, opinion_term, sentiment, aspect_category = postprocessor.post_process(entity, aspect_term, opinion_term, sentiment, aspect_category, orig_sentence)
        aspect_term = aspect_term.replace(',','')
        opinion_term = opinion_term.replace(',','')
        clean_quintuplets.append(f'({entity},{aspect_term},{opinion_term},{sentiment},{aspect_category})')
    df.at[i, 'postprocess_quintuplet'] = ';'.join(clean_quintuplets)

In [8]:
errors_quad = []
for i in range(len(df)):
    row = df.iloc[i]
    quintuplets = extract_quintuplet(row['postprocess_quintuplet'])
    for quintuplet in quintuplets:
        entity, aspect_term, opinion_term, sentiment, aspect_category = quintuplet
        if entity=='' or aspect_term=='' or opinion_term=='' or sentiment=='' or aspect_category=='':
            print(i, quintuplet)
            errors_quad.append(i)
errors_quad

[]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   aoriginal_id              100 non-null    int64 
 1   content                   100 non-null    object
 2   final_sentiment           100 non-null    object
 3   baseline_aspect_category  100 non-null    object
 4   tweet_type                100 non-null    object
 5   clean_tweet               100 non-null    object
 6   label                     100 non-null    object
 7   Unnamed: 7                1 non-null      object
 8   postprocess_quintuplet    100 non-null    object
dtypes: int64(1), object(8)
memory usage: 7.2+ KB


In [10]:
df.head(3)

,aoriginal_id,content,final_sentiment,baseline_aspect_category,tweet_type,clean_tweet,label,Unnamed: 7,postprocess_quintuplet
0,1648753333681917954,@JUALAN_BASE @fleurlovincs Nder hpnya kamu pak...,neutral,delivery; produk;,SUBJECTIVE,nder hpnya kamu pake brp lama? kayaknya jt msh...,"(olx, _, banyak yg jual dibawah 4jt, positive,...",NaN,"(olx,_,banyak yg jual dibawah jt,positive,price)"
1,1647741896079589377,@tanyakanrl Kalo mau beli second jangan di tok...,positive,delivery; produk;,SUBJECTIVE,kalo mau beli second jangan di toko oren atau ...,"(olx, _, lebih aman, positive, website&apps);(...",NaN,"(olx,_,lebih aman,positive,website&apps);(fb,m..."
2,1647638497573085184,"@tirta_cipeng Di luar dunia fashion, saya seri...",neutral,produk;,SUBJECTIVE,"di luar dunia fashion, saya sering banget nemu...","(olx, reseller, ga jujur, negative, other);(ol...",NaN,"(olx,reseller,ga jujur,negative,price);(olx,ma..."


In [11]:
df.to_csv('Data/quadruplet/quintuplet_100_data_clean.csv', index=False)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   aoriginal_id              1000 non-null   float64
 1   content                   1000 non-null   object 
 2   final_sentiment           1000 non-null   object 
 3   baseline_aspect_category  1000 non-null   object 
 4   tweet_type                1000 non-null   object 
 5   clean_tweet               1000 non-null   object 
 6   quadruplet                998 non-null    object 
 7   corrected_quadruplet      1000 non-null   object 
 8   review MLK                8 non-null      object 
 9   is_comparative            1000 non-null   bool   
 10  postprocess_quad          1000 non-null   object 
dtypes: bool(1), float64(1), object(9)
memory usage: 119.2+ KB


In [23]:
df.iloc[[403, 703]]

,aoriginal_id,content,final_sentiment,baseline_aspect_category,tweet_type,clean_tweet,quadruplet,corrected_quadruplet,postprocess_quad
403,1.647931e+18,Selama ngurus barang retur. Dari semua platfor...,positive,produk;,SUBJECTIVE,selama ngurus barang retur. dari semua platfor...,"(barang, cuman yang paling cepet dah, positive...","barang retur, yang paling cepet dah, positive,...",
703,1.647237e+18,verifikasi qr nya shopee nyusahin banget,negative,produk;,SUBJECTIVE,verifikasi qr nya shopee nyusahin banget,"verifikasi qr shopee, nyusahin banget, negativ...","verifikasi qr shopee, nyusahin banget, negativ...",


# Quintuplet Data

In [5]:
import pandas as pd
import numpy as np
from experiments.src.postprocessor import PostProcessor
from experiments.src.utils import extract_quintuplet

In [42]:
df = pd.read_csv('Data/quintuplet/quintuple_data_clean.csv')

In [43]:
df['quintuplet_label'] = df.apply(lambda x: x.corrected_label if str(x.corrected_label)!='nan' else x.label, axis=1)

In [44]:
df = df.dropna(subset=['quintuplet_label'])
df = df.reset_index(drop=True)

In [45]:
postprocessor = PostProcessor(use_postprocess=False)

In [46]:
df['postprocess_quintuplet']=''
for i in range(len(df)):
    row = df.iloc[i]
    orig_sentence = row['clean_tweet']
    clean_quintuplets = []
    quintuplets = extract_quintuplet(row['quintuplet_label'])
    for quintuplet in quintuplets:
        entity, aspect_term, opinion_term, sentiment, aspect_category = quintuplet
        _, _, _, sentiment, aspect_category = postprocessor.post_process(entity, aspect_term, opinion_term, sentiment, aspect_category, orig_sentence)
        clean_quintuplets.append(f'({entity},{aspect_term},{opinion_term},{sentiment},{aspect_category})')
    df.at[i, 'postprocess_quintuplet'] = ';'.join(clean_quintuplets)

In [47]:
errors_quad = []
for i in range(len(df)):
    row = df.iloc[i]
    quintuplets = extract_quintuplet(row['postprocess_quintuplet'])
    for quintuplet in quintuplets:
        entity, aspect_term, opinion_term, sentiment, aspect_category = quintuplet
        if entity=='' or aspect_term=='' or opinion_term=='' or sentiment=='' or aspect_category=='':
            print(i, quintuplet)
            errors_quad.append(i)
errors_quad

[]

In [41]:
i = 1643
print(df.iloc[i]['content'])
print(df.iloc[i]['quintuplet_label'])
print(extract_quintuplet(df.iloc[i]['quintuplet_label']))
print(extract_quintuplet(df.iloc[i]['postprocess_quintuplet']))

@tokopedia min top up tapcash @BNICustomerCare lagi bermasalah ya? dari tadi pending gabisa update saldo
(_ top up tapcash, pending gabisa update saldo, negative, website&apps)
[('', '', '', '', '')]
[('', '', '', 'neutral', 'price')]


In [48]:
df.to_csv('Data/quintuplet/quintuplet_postprocessed.csv', index=False)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   aoriginal_id              2001 non-null   int64 
 1   content                   2001 non-null   object
 2   final_sentiment           2001 non-null   object
 3   baseline_aspect_category  1973 non-null   object
 4   tweet_type                2001 non-null   object
 5   clean_tweet               2001 non-null   object
 6   label                     2001 non-null   object
 7   corrected_label           182 non-null    object
 8   keterangan                53 non-null     object
 9   quintuplet_label          2001 non-null   object
 10  postprocess_quintuplet    2001 non-null   object
dtypes: int64(1), object(10)
memory usage: 172.1+ KB


In [134]:
0.9*len(df)

930.6

In [50]:
df2=pd.read_csv('Data/quintuplet/quintuplet_postprocessed.csv')

In [51]:
df2.iloc[120]['postprocess_quintuplet']

nan

In [55]:
for i in range(len(df2)):
    try:
        if 'costumerservice' in df2.iloc[i]['postprocess_quintuplet']:
            print(i)
    except Exception as e:
        print('error', i, e)

error 120 argument of type 'float' is not iterable
error 135 argument of type 'float' is not iterable
error 146 argument of type 'float' is not iterable
error 162 argument of type 'float' is not iterable
error 174 argument of type 'float' is not iterable
error 175 argument of type 'float' is not iterable
error 183 argument of type 'float' is not iterable


In [53]:
df2.iloc[120]

aoriginal_id                                              1648075525968248832
content                     @banjarbase Mahal😭 aku tim shopee krinkle soal...
final_sentiment                                                      negative
baseline_aspect_category                                              produk;
tweet_type                                                         SUBJECTIVE
clean_tweet                              Mahal aku tim shopee krinkle soalnya
label                        (shopee, shopee krinkle, mahal, negative, price)
corrected_label             kayanya ini gk usah dimasukin zil, gk jelas gi...
keterangan                                                                NaN
quintuplet_label            kayanya ini gk usah dimasukin zil, gk jelas gi...
postprocess_quintuplet                                                    NaN
Name: 120, dtype: object

In [54]:
quintuplets = extract_quintuplet(df2.iloc[120]['quintuplet_label'])
for quintuplet in quintuplets:
    entity, aspect_term, opinion_term, sentiment, aspect_category = quintuplet
    print(aspect_category)
    _, _, _, sentiment, aspect_category = postprocessor.post_process(entity, aspect_term, opinion_term, sentiment, aspect_category, orig_sentence)
    print(aspect_category)